In [ ]:
!sudo apt-get install zstd
!curl -fsSL https://ollama.com/install.sh | sh

In [ ]:
# Run this on terminal.
ollama serve

In [ ]:
!ollama pull llama3.1

In [ ]:
!ollama list

In [ ]:
!pip install crewai
!pip install -U litellm
!pip install crewai_tools
!pip install -U langchain-ollama langchain-community
!pip install -q gradio
!pip install -U huggingface_hub diffusers transformers accelerate

## **v1.0 - Initial release with three-agent system:**

In [ ]:
from crewai import Agent, Task, Crew, LLM
from langchain_community.llms import Ollama
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
import os


os.environ["OPENAI_API_KEY"] = "NA"
MODEL_NAME = "llama3.1:latest"

llm = LLM(
    model="ollama/llama3.1", # The prefix 'ollama/' is mandatory here
    base_url="http://localhost:11434",
    # api_key="NA" # Sometimes required to stop the OpenAI check
)

# Initialize tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Agent 1: Trend Analyzer
trend_agent = Agent(
    role="Social Media Trend Analyst",
    goal="Identify trending topics and viral patterns in social media",
    backstory="Expert at spotting viral patterns and analyzing social media trends across platforms",
    llm=llm,
    tools=[search_tool, scrape_tool],
    verbose=True,
    allow_delegation=False
)

# Agent 2: Brand Strategist
strategy_agent = Agent(
    role="Brand Strategy Consultant",
    goal="Create cohesive brand voice and content pillars based on trend analysis",
    backstory="Specializes in startup positioning and building authentic brand narratives",
    llm=llm,
    tools=[search_tool],
    verbose=True,
    allow_delegation=False
)

# Agent 3: Content Creator
content_agent = Agent(
    role="Social Media Content Creator",
    goal="Generate engaging social media posts that align with brand strategy",
    backstory="Creative writer skilled at crafting viral-worthy content that resonates with audiences",
    llm=llm,
    verbose=True,
    allow_delegation=False
)

# Task 1: Research Trends
research_task = Task(
    description="""
    Research current trending topics in {industry} on social media platforms.
    Focus on:
    - Popular hashtags and topics
    - Viral content formats
    - Audience engagement patterns
    - Competitor content strategies

    Provide a comprehensive report with at least 5 key trends.
    """,
    expected_output="A detailed report listing 5+ trending topics with explanations and examples",
    agent=trend_agent
)

# Task 2: Develop Brand Strategy
strategy_task = Task(
    description="""
    Based on the trend research, develop a brand strategy for {company_name}.
    Create:
    - Brand voice guidelines (tone, style, personality)
    - 3-5 content pillars aligned with trends
    - Target audience insights
    - Unique positioning angle

    Ensure the strategy feels authentic and differentiates from competitors.
    """,
    expected_output="A comprehensive brand strategy document with voice guidelines and content pillars",
    agent=strategy_agent,
    context=[research_task]  # This task depends on research_task
)

# Task 3: Create Content
content_task = Task(
    description="""
    Create 5 engaging social media posts for {company_name} based on the brand strategy.
    For each post include:
    - Platform (Twitter/LinkedIn/Instagram)
    - Post copy (with appropriate length for platform)
    - Suggested hashtags
    - Content type (text, image idea, video idea)
    - Best posting time recommendation

    Make posts feel natural, not overly promotional.
    """,
    expected_output="5 complete social media posts with all required elements",
    agent=content_agent,
    context=[strategy_task]  # This task depends on strategy_task
)

# Create Crew
crew = Crew(
    agents=[trend_agent, strategy_agent, content_agent],
    tasks=[research_task, strategy_task, content_task],
    verbose=False
)

# Execute the crew
result = crew.kickoff(inputs={
    'industry': 'sustainable fashion',  # Change this to your industry
    'company_name': 'EcoThreads'  # Change this to your company name
})

print("\n\n========================")
print("FINAL RESULT:")
print("========================")
print(result)

## **v2.0 - Gradio UI implementation**

In [ ]:
import os
import gradio as gr
from crewai import Agent, Task, Crew, LLM
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# 1. Setup Environment
os.environ["OPENAI_API_KEY"] = "NA"
# Ensure your SERPER_API_KEY is set if you're using SerperDevTool
os.environ["SERPER_API_KEY"] = "7efe5d51c88e3e9d76a790c7f420faa8ffc777d6"

# 2. Initialize LLM (Ollama)
llm = LLM(
    model="ollama/llama3.1",
    base_url="http://localhost:11434"
)

# 3. Define the Function for Gradio
def run_crew_manager(industry, company_name):
    """
    This function takes user input from Gradio and executes the CrewAI process.
    """
    if not industry or not company_name:
        return "Please provide both an Industry and a Company Name."

    # Initialize tools
    search_tool = SerperDevTool()
    scrape_tool = ScrapeWebsiteTool()

    # Define Agents
    trend_agent = Agent(
        role="Social Media Trend Analyst",
        goal=f"Identify trending topics and viral patterns in {industry}",
        backstory="Expert at spotting viral patterns and analyzing social media trends.",
        llm=llm,
        tools=[search_tool, scrape_tool],
        verbose=True
    )

    strategy_agent = Agent(
        role="Brand Strategy Consultant",
        goal=f"Create cohesive brand voice for {company_name} based on trends",
        backstory="Specializes in startup positioning and building narratives.",
        llm=llm,
        tools=[search_tool],
        verbose=True
    )

    content_agent = Agent(
        role="Social Media Content Creator",
        goal=f"Generate engaging posts for {company_name} that align with strategy",
        backstory="Creative writer skilled at crafting viral-worthy content.",
        llm=llm,
        verbose=True
    )

    # Define Tasks
    research_task = Task(
        description=f"Research current trending topics in {industry} on social media. Focus on hashtags and viral formats.",
        expected_output="A detailed report listing 5+ trending topics.",
        agent=trend_agent
    )

    strategy_task = Task(
        description=f"Develop a brand strategy for {company_name} using the research.",
        expected_output="A strategy document with voice guidelines and content pillars.",
        agent=strategy_agent,
        context=[research_task]
    )

    content_task = Task(
        description=f"Create 5 engaging social media posts for {company_name} for Twitter, LinkedIn, and Instagram.",
        expected_output="5 complete social media posts with hashtags and timing recommendations.",
        agent=content_agent,
        context=[strategy_task]
    )

    # Create and Run Crew
    crew = Crew(
        agents=[trend_agent, strategy_agent, content_agent],
        tasks=[research_task, strategy_task, content_task],
        verbose=True
    )

    # Execute
    result = crew.kickoff(inputs={
        'industry': industry,
        'company_name': company_name
    })

    return str(result)
# 4. Build Gradio UI
# gr.Interface follows a strict Input -> Function -> Output pattern.
# You don't have to manually define rows, columns, or click events; Gradio handles the UI generation for you.
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🤖 AI Social Media Manager")
    gr.Markdown("Enter your details below to start the multi-agent research and content creation process.")

    with gr.Row():
        with gr.Column():
            input_industry = gr.Textbox(label="Industry", placeholder="e.g., Sustainable Fashion")
            input_company = gr.Textbox(label="Company Name", placeholder="e.g., EcoThreads")
            run_btn = gr.Button("🚀 Generate Strategy & Posts", variant="primary")

    output_area = gr.Markdown(label="Final Result")

    # Link button to function
    run_btn.click(
        fn=run_crew_manager,
        inputs=[input_industry, input_company],
        outputs=output_area
    )

# 5. Launch
demo.launch(share=True)

# **v2.1 - Added Stable Diffusion image generation**

In [ ]:
import os
import gradio as gr
import torch
from crewai import Agent, Task, Crew, LLM
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from diffusers import StableDiffusionPipeline
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# ENVIRONMENT & MODEL SETUP
# ==============================================================================

os.environ["OPENAI_API_KEY"] = "NA"
SERPER_API_KEY = os.getenv("SERPER_API_KEY", "")
if SERPER_API_KEY:
    os.environ["SERPER_API_KEY"] = SERPER_API_KEY

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize LLM
try:
    local_llm = LLM(
        model="ollama/llama3.1",
        base_url="http://localhost:11434"
    )
    print("✓ Ollama LLM initialized")
except Exception as e:
    print(f"⚠️ Error initializing Ollama: {e}")
    local_llm = None

# Initialize Stable Diffusion
sd_pipeline = None
try:
    print("\nLoading Stable Diffusion v1.4 (~2GB)...")

    sd_pipeline = StableDiffusionPipeline.from_pretrained(
        "CompVis/stable-diffusion-v1-4",
        torch_dtype=torch.float16 if device == "cuda" else torch.float32,
        safety_checker=None  # Disable for speed since it is a demo.
    ).to(device)

    # Enable memory optimizations
    # Instead of calculating the entire "attention matrix" (the part of the AI that connects words to pixels) all at once,
    # It breaks the calculation into smaller, sequential steps (slices).
    sd_pipeline.enable_attention_slicing()

    print("✓ Stable Diffusion loaded successfully\n")

except Exception as e:
    print(f"⚠️ Error loading Stable Diffusion: {e}\n")

# ==============================================================================
# MAIN FUNCTION
# ==============================================================================

def run_social_manager_with_image(industry, company_name, progress=gr.Progress()):
    try:
        if not industry or not company_name:
            return "⚠️ Please provide both Industry and Company Name.", None, "No output"

        if local_llm is None:
            return """⚠️ **Error: Ollama LLM not initialized**

Please ensure:
1. Ollama is running: `ollama serve`
2. llama3.1 model is pulled: `ollama pull llama3.1`
            """, None, "LLM not available"

        # --- CrewAI Logic ---
        progress(0.1, desc="🚀 Initializing Agents...")

        tools_list = []
        if SERPER_API_KEY:
            try:
                search_tool = SerperDevTool()
                tools_list.append(search_tool)
            except Exception as e:
                print(f"Warning: Could not initialize SerperDevTool: {e}")

        try:
            scrape_tool = ScrapeeWbsiteTool()
        except Exception as e:
            scrape_tool = None

        # Agents
        trend_agent = Agent(
            role="Social Media Trend Analyst",
            goal=f"Identify trending topics in {industry} for {company_name}",
            backstory="Expert at spotting viral patterns.",
            llm=local_llm,
            tools=[scrape_tool] if scrape_tool else [],
            verbose=False
        )

        strategy_agent = Agent(
            role="Brand Strategy Consultant",
            goal=f"Create brand voice for {company_name} in {industry}",
            backstory="Startup positioning expert.",
            llm=local_llm,
            tools=tools_list if tools_list else [],
            verbose=False
        )

        content_agent = Agent(
            role="Social Media Content Creator",
            goal=f"""Generate 5 social media posts for {company_name} and a detailed image prompt.""",
            backstory="Creative writer skilled at viral content and minimalist design.",
            llm=local_llm,
            verbose=False
        )

        # Tasks
        progress(0.3, desc="🔍 Researching Trends...")
        research_task = Task(
            description=f"Research current trending topics in {industry}.",
            expected_output="A report listing 5+ trending topics.",
            agent=trend_agent
        )

        progress(0.5, desc="📋 Developing Strategy...")
        strategy_task = Task(
            description=f"Develop brand strategy for {company_name} using trends.",
            expected_output="Brand strategy with voice guidelines.",
            agent=strategy_agent,
            context=[research_task]
        )

        progress(0.7, desc="✍️ Generating Content...")
        content_task = Task(
            description=f"""
Create 5 engaging social media posts for {company_name}.
Include: Platform, Post copy, Hashtags, Content type, Best time.

Generate ONE image prompt at the end:

---
IMAGE PROMPT:
[Professional minimalist logo for {company_name}, {industry}, modern, clean, NO TEXT, NO WORDS]
            """,
            expected_output="5 social media posts followed by image prompt.",
            agent=content_agent,
            context=[strategy_task]
        )

        crew = Crew(
            agents=[trend_agent, strategy_agent, content_agent],
            tasks=[research_task, strategy_task, content_task],
            verbose=False
        )

        crew_result_raw = crew.kickoff(inputs={'industry': industry, 'company_name': company_name})
        crew_result_str = str(crew_result_raw)

        # --- Extract image prompt ---
        image_prompt = f"Professional minimalist logo for {company_name}, {industry} company, modern geometric design, clean, abstract, no text, no words"
        social_posts_text = crew_result_str

        for marker in ["IMAGE PROMPT:", "Image Prompt:", "image prompt:"]:
            if marker in crew_result_str:
                parts = crew_result_str.split(marker, 1)
                social_posts_text = parts[0].strip()
                image_prompt = parts[1].strip()

                if "no text" not in image_prompt.lower():
                    image_prompt += ", no text, no words"
                break

        # --- Generate Image ---
        generated_image = None
        if sd_pipeline is not None:
            try:
                progress(0.9, desc="🖼️ Generating Image...")

                generated_image = sd_pipeline(
                    prompt=image_prompt,
                    num_inference_steps=20,  # number of times the model refines the image from random noise.
                    guidance_scale=7.5, # controls how closely the model follows your prompt.
                    height=512,
                    width=512
                ).images[0]

            except Exception as e:
                print(f"Error generating image: {e}")
                image_prompt += f"\n\n⚠️ Error: {str(e)}"
        else:
            image_prompt += "\n\n⚠️ Image generation not available"

        progress(1.0, desc="✅ Complete!")
        return social_posts_text, generated_image, image_prompt

    except Exception as e:
        error_msg = f"⚠️ **Error:**\n\n```\n{str(e)}\n```"
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()
        return error_msg, None, str(e)

# ==============================================================================
# GRADIO UI
# ==============================================================================
# using soft theme - a pre-built theme.
with gr.Blocks(theme=gr.themes.Soft(), title="AI Social Media Manager") as demo:
    gr.Markdown("# 🤖 AI Social Media Manager")
    gr.Markdown("### Generate social media posts and visuals with AI")

    # Status
    status = []
    if local_llm:
        status.append("✓ LLM Ready")
    else:
        status.append("⚠️ LLM Not Available")

    if sd_pipeline:
        status.append("✓ Image Generation Ready (SD v1.4 - 2GB)")
    else:
        status.append("⚠️ Image Generation Disabled")

    gr.Markdown(f"**Status:** {' | '.join(status)}")

    with gr.Row():
        with gr.Column(scale=1):
            industry_input = gr.Textbox(
                label="Target Industry",
                placeholder="e.g., AI Smart Homes, Sustainable Fashion",
                value="AI smart homes"
            )
            company_input = gr.Textbox(
                label="Company Name",
                placeholder="e.g., TechLiving, EcoThreads",
                value="TechLiving"
            )
            generate_btn = gr.Button("🚀 Generate Content & Image", variant="primary")

            gr.Markdown("""
            ### Requirements:
            - Ollama running (`ollama serve`)
            - llama3.1 model (`ollama pull llama3.1`)
            """)

            image_prompt_display = gr.Textbox(
                label="Generated Image Prompt",
                interactive=False,
                lines=3
            )

        with gr.Column(scale=2):
            image_output = gr.Image(label="Generated Visual", type="pil", height=512)
            social_posts_output = gr.Markdown(label="Social Media Posts")

    generate_btn.click(
        fn=run_social_manager_with_image,
        inputs=[industry_input, company_input],
        outputs=[social_posts_output, image_output, image_prompt_display]
    )

if __name__ == "__main__":
    demo.launch(share=True)